In [1]:
import json
import pandas as pd
from pyspark import SparkContext
from pyspark.rdd import RDD

In [2]:
sc = SparkContext(appName="SubsetJSON")

24/11/16 15:48:27 WARN Utils: Your hostname, Adams-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.0.0.82 instead (on interface en0)
24/11/16 15:48:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/16 15:48:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/16 15:48:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
meta_subset_file = '../subset_meta.jsonl'
subset_file = '../subsetAF.jsonl'
data = []

with open(subset_file, 'r') as fp:
    for i, line in enumerate(fp):
        data.append(json.loads(line))

metadata = []
with open(meta_subset_file, 'r') as fp:
    for i, line in enumerate(fp):
        metadata.append(json.loads(line))

In [4]:
subset_rdd = sc.parallelize(data)
meta_subset_rdd = sc.parallelize(metadata)

In [37]:
meta_rdd = meta_subset_rdd.map(lambda review: list(review.values()))
meta_pair_rdd = meta_rdd.map(lambda row: (row[-2], row[:-2] + [row[-1]]))


In [5]:
values_rdd = subset_rdd.map(lambda review: list(review.values()))

In [14]:
pair_rdd = values_rdd.map(lambda row: (row[5], row[:5] + row[6:]))

In [7]:
avg_stars = {}
for i in range(5):
    avg_stars[i+1] = values_rdd.filter(lambda row: row[0] == float(i+1)).map(lambda row: len(row[2])).mean()

In [8]:
avg_stars


{1: 241.24924012158056,
 2: 339.53333333333336,
 3: 343.615969581749,
 4: 389.0729261622608,
 5: 283.26746290264197}

In [50]:
joined_rdd = meta_pair_rdd.join(pair_rdd)
# first element in the value is the product metadata
# second element in the value is the review
# right now we only have 76 matches, because we are working with a subset of both metadata and the reviews. 
joined_rdd.count()


24/11/16 16:05:51 WARN TaskSetManager: Stage 75 contains a task of very large size (1022 KiB). The maximum recommended task size is 1000 KiB.


76